In [1]:
# import utility packages
from tqdm import tqdm
from time import time

In [2]:
# import torch packages
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision

In [3]:
# import torch summanry (like model.summary() in keras)
from torchsummary import summary

In [4]:
# import einops packages
from einops.layers.torch import Rearrange

In [5]:
# Setting hyperparameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device {}'.format(device))
n_epochs = 3
batch_size_train = 64
batch_size_test = 64
learning_rate = 0.01
momentum = 0.5
log_interval = 10
if device == torch.device('cuda'):
    random_seed = 42
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
else:
    random_seed = 42
    torch.backends.cudnn.enabled = False
    torch.manual_seed(random_seed)

Using device cpu


In [6]:
# Loading data
train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('files/', train=True, download=True,
                               transform=torchvision.transforms.Compose([
                                   torchvision.transforms.ToTensor(),
                                   torchvision.transforms.Normalize(
                                       (0.1307,), (0.3081,))
                               ])),
    batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('files/', train=False, download=True,
                               transform=torchvision.transforms.Compose([
                                   torchvision.transforms.ToTensor(),
                                   torchvision.transforms.Normalize(
                                       (0.1307,), (0.3081,))
                               ])),
    batch_size=batch_size_test, shuffle=True)

In [15]:
# Traditional model using Pytorch API
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [15]:
from layers import EinConv2d, EinMaxPool2d, EinLinear
from layers import PureEinConv2d

# My model using alternative einsum layers
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = EinConv2d(1, 10, kernel_size=5)
        self.conv2 = EinConv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = EinLinear(320, 1024)
        self.fc2 = EinLinear(1024, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = EinMaxPool2d(2)(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.conv2_drop(x)
        x = EinMaxPool2d(2)(x)
        x = F.relu(x)
        x = Rearrange('b c h w -> b (c h w)')(x)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [16]:
# Trainer
def train(epoch, total_epoch):
    network.to(device).train()
    print('Train Epoch {} of {}'.format(epoch, total_epoch))
    t = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(t):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            train_losses.append(loss.item())
            t.set_postfix({'Loss': sum(train_losses)/len(train_losses)})
            train_counter.append(batch_idx*64 + (epoch + 1)
                                 * len(train_loader.dataset))
    torch.save(network.state_dict(), 'result/model.pth')
    torch.save(optimizer.state_dict(), 'result/optimizer.pth')

# Tester
def test():
    network.to(device).eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        test_loss /= len(test_loader.dataset)
        test_losses.append(test_loss)
        print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

In [17]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [18]:
summary(network, (1, 28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
├─EinConv2d: 1-1                         [-1, 10, 24, 24]          260
├─EinConv2d: 1-2                         [-1, 20, 8, 8]            5,020
├─Dropout2d: 1-3                         [-1, 20, 8, 8]            --
├─EinLinear: 1-4                         [-1, 1024]                328,704
├─EinLinear: 1-5                         [-1, 10]                  10,250
Total params: 344,234
Trainable params: 344,234
Non-trainable params: 0
Total mult-adds (M): 0.80
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 1.31
Estimated Total Size (MB): 1.38


Layer (type:depth-idx)                   Output Shape              Param #
├─EinConv2d: 1-1                         [-1, 10, 24, 24]          260
├─EinConv2d: 1-2                         [-1, 20, 8, 8]            5,020
├─Dropout2d: 1-3                         [-1, 20, 8, 8]            --
├─EinLinear: 1-4                         [-1, 1024]                328,704
├─EinLinear: 1-5                         [-1, 10]                  10,250
Total params: 344,234
Trainable params: 344,234
Non-trainable params: 0
Total mult-adds (M): 0.80
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 1.31
Estimated Total Size (MB): 1.38

In [14]:
test()
total_training_time = 0
for epoch in range(1, n_epochs + 1):
    start = time()
    train(epoch, n_epochs)
    end = time()
    print('\nTraning process takes {} seconds'.format(end - start))
    total_training_time += end - start
    test()

  0%|          | 2/938 [00:00<00:54, 17.24it/s, Loss=2.33]
Test set: Avg. loss: 2.3060, Accuracy: 1067/10000 (11%)

Train Epoch 1 of 3
100%|██████████| 938/938 [00:50<00:00, 18.41it/s, Loss=0.662]

Traning process takes 50.98199987411499 seconds
  0%|          | 2/938 [00:00<01:02, 15.03it/s, Loss=0.658]
Test set: Avg. loss: 0.1489, Accuracy: 9542/10000 (95%)

Train Epoch 2 of 3
100%|██████████| 938/938 [00:48<00:00, 19.27it/s, Loss=0.448]

Traning process takes 48.69596719741821 seconds
  0%|          | 2/938 [00:00<00:53, 17.39it/s, Loss=0.446]
Test set: Avg. loss: 0.0866, Accuracy: 9737/10000 (97%)

Train Epoch 3 of 3
100%|██████████| 938/938 [00:46<00:00, 20.03it/s, Loss=0.358]

Traning process takes 46.83997178077698 seconds

Test set: Avg. loss: 0.0685, Accuracy: 9789/10000 (98%)

